Gauname nuotraukų

In [18]:
!pip install openimages

In [19]:
import os
from openimages.download import download_dataset

In [20]:
data_dir = "OpenImages"
number_for_samples = 333
classes = ["Strawberry", "Bee", "Goldfish"]

In [21]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [22]:
print("Downloading is starting...")
download_dataset(data_dir, classes, limit=number_for_samples)

100%|██████████| 333/333 [00:21<00:00, 15.20it/s]


{'strawberry': {'images_dir': 'OpenImages/strawberry/images'},
 'bee': {'images_dir': 'OpenImages/bee/images'},
 'goldfish': {'images_dir': 'OpenImages/goldfish/images'}}

Paruošiam transformacijos parametrus

In [23]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Custom Dataset Class

In [24]:
from torch.utils.data import Dataset
from PIL import Image

class GMMDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Custom dataset class to load images from 'data_dir/class_name/images/' structure.

        Parameters:
        - data_dir (str): Path to the dataset directory.
        - transform (callable, optional): Transformations to apply to the images.
        """
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Define class-to-index mapping (case insensitive)
        self.class_to_idx = {"strawberry": 0, "bee": 1, "goldfish": 2}

        # Scan the dataset structure correctly
        for class_name, class_idx in self.class_to_idx.items():
            class_path = os.path.join(data_dir, class_name, "images")  # Look inside 'images' subfolder
            if os.path.exists(class_path):
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(class_idx)

        print(f"Loaded {len(self.image_paths)} images from {data_dir}")

    def __len__(self):
        """Returns the number of images in the dataset."""
        return len(self.image_paths)

    def __getitem__(self, idx):
        """Loads an image and its label."""
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Open image and convert to RGB
        image = Image.open(img_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, label



Create a dataset and put it in a dataloader



In [25]:
from torch.utils.data import DataLoader
dataset = GMMDataset(data_dir="OpenImages", transform=transform)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers = 2)
iterator = iter(dataloader)

Loaded 999 images from OpenImages


Paruošiam pre-trained modelį darbui

In [26]:
import torch
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg19(pretrained=True)
model.eval().to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

Skaičiavimai

In [28]:
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



# Define selected class indices
STRAWBERRY_IDX = 949  # Strawberry
BEE_IDX = 309  # Bee
GOLDFISH_IDX = 1  # Zebra

ground_truth = []
predictions_strawberry = []
predictions_bee = []
predictions_goldfish = []

# Perform inference
while True:
    try:
        features, labels = next(iterator)
        output = model(features.to(device))

        for i in range(output.shape[0]):
            predictions = torch.softmax(output[i], 0)
            predictions_strawberry = np.append(predictions_strawberry, predictions[STRAWBERRY_IDX].cpu().detach())
            predictions_bee = np.append(predictions_bee, predictions[BEE_IDX].cpu().detach())
            predictions_goldfish = np.append(predictions_goldfish, predictions[GOLDFISH_IDX].cpu().detach())

        ground_truth = np.append(ground_truth, labels)
    except StopIteration:
        break

# Compute confusion matrix
def calculate_confusion_matrix(ground_truth, predictions, class_idx, threshold=0.5):
    predictions = np.array(predictions)  # Convert to NumPy array
    predictions_thresholded = (predictions >= threshold).astype(np.float64)

    matrix = {
        'TP': np.sum((ground_truth == class_idx) & (predictions_thresholded == 1)),
        'TN': np.sum((ground_truth != class_idx) & (predictions_thresholded == 0)),
        'FP': np.sum((ground_truth != class_idx) & (predictions_thresholded == 1)),
        'FN': np.sum((ground_truth == class_idx) & (predictions_thresholded == 0)),
    }
    return matrix


# Compute metrics
def calculate_metrics(TP, TN, FP, FN):
    metrics = {}
    metrics['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
    metrics['recall'] = TP / (TP + FN) if (TP + FN) > 0 else 0
    metrics['precision'] = TP / (TP + FP) if (TP + FP) > 0 else 0
    metrics['f1'] = 2 * (metrics['recall'] * metrics['precision']) / (metrics['recall'] + metrics['precision']) if (metrics['recall'] + metrics['precision']) else 0
    return metrics

# Evaluate for each class
matrix_strawberry = calculate_confusion_matrix(ground_truth, predictions_strawberry, 0, 0.05)
matrix_bee = calculate_confusion_matrix(ground_truth, predictions_bee, 1, 0.05)
matrix_goldfish = calculate_confusion_matrix(ground_truth, predictions_goldfish, 2, 0.05)

metrics_strawberry = calculate_metrics(matrix_strawberry['TP'], matrix_strawberry['TN'], matrix_strawberry['FP'], matrix_strawberry['FN'])
metrics_bee = calculate_metrics(matrix_bee['TP'], matrix_bee['TN'], matrix_bee['FP'], matrix_bee['FN'])
metrics_goldfish = calculate_metrics(matrix_goldfish['TP'], matrix_goldfish['TN'], matrix_goldfish['FP'], matrix_goldfish['FN'])

# Print results
def print_metrics(metrics, class_name):
    print(f'Class {class_name} Metrics:')
    print(f'  Accuracy : {metrics["accuracy"]:.2f}')
    print(f'  Recall   : {metrics["recall"]:.2f}')
    print(f'  Precision: {metrics["precision"]:.2f}')
    print(f'  F1-score : {metrics["f1"]:.2f}\n')


print_metrics(metrics_strawberry, "Strawberry")
print_metrics(metrics_bee, "Bee")
print_metrics(metrics_goldfish, "Goldfish")

Class Strawberry Metrics:
  Accuracy : nan
  Recall   : 0.00
  Precision: 0.00
  F1-score : 0.00

Class Bee Metrics:
  Accuracy : nan
  Recall   : 0.00
  Precision: 0.00
  F1-score : 0.00

Class Goldfish Metrics:
  Accuracy : nan
  Recall   : 0.00
  Precision: 0.00
  F1-score : 0.00

